In [23]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import cohen_kappa_score, mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import spearmanr
from scipy.optimize import minimize
import os
import joblib

In [24]:
### Define targets
ordinal_targets = ['ADNC', 'Braak', 'Thal', 'CERAD']
continuous_targets = ['percent 6e10 positive area', 'percent AT8 positive area', 'percent GFAP positive area']
                      #'percent NeuN positive area'

In [25]:
### Train final model
def train_final_model(df, target, region, random_state=42):
    
    ## Prepare data (create feature matrix (x) and target vectors (y))
    X, y = df.drop(columns=['Donor ID', target]), df[target].values
    
    ## Train model (XGBoost regressor)
    model = xgb.XGBRegressor(random_state=random_state, eval_metric='rmse')
    model.fit(X, y)
    y_pred = model.predict(X)
    
    ## Calculate metrics based on target type (according to DREAM tasks)
    if target in ordinal_targets:
        # Optimize thresholds for ordinal targets and calculate metrics
        class_props = pd.Series(y).value_counts(normalize=True).sort_index()
        initial_th = np.quantile(y_pred, q=np.cumsum(class_props).iloc[:-1])
        
        result = minimize(
            lambda th: -cohen_kappa_score(y, np.digitize(y_pred, bins=np.sort(th)), weights='quadratic'),
            x0=initial_th, method='Nelder-Mead'
        )
        
        y_classes = np.digitize(y_pred, bins=np.sort(result.x))
        metrics = {
            'quadratic_weighted_kappa': cohen_kappa_score(y, y_classes, weights='quadratic'),
            'mean_absolute_error': mean_absolute_error(y, y_classes),
            'spearman_correlation': spearmanr(y, y_classes)[0],
            'optimized_thresholds': str(np.sort(result.x).round(3))
        }

    else:
        # Calculate metrics for continuous targets
        mean_t, mean_p = np.mean(y), np.mean(y_pred)
        var_t, var_p = np.var(y, ddof=1), np.var(y_pred, ddof=1)
        ccc = (2 * np.cov(y, y_pred)[0,1]) / (var_t + var_p + (mean_t - mean_p)**2)
        
        metrics = {
            'concordance_correlation_coefficient': ccc,
            'mean_squared_error': mean_squared_error(y, y_pred),
            'r2_score': r2_score(y, y_pred)
        }
    
    ### Save model
    joblib.dump(model, f'./output/{region}/{target}_model.pkl')
    
    ### Save metrics
    metrics_df = pd.DataFrame(list(metrics.items()), columns=['metric_name', 'metric_value'])
    metrics_df.to_csv(f'./output/{region}/{target}_metrics.csv', index=False)
    
    ### Save feature importance
    importance_df = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
    importance_df.sort_values('importance', ascending=False).to_csv(
        f'./output/{region}/{target}_feature_importance.csv', index=False)
    
    return model

In [26]:
### Process all datasets
for csv_file in os.listdir('data/'):
    if csv_file in ['dataset_mtg.csv', 'dataset_a9.csv']:
        
        region = 'MTG' if 'mtg' in csv_file else 'A9'
        df = pd.read_csv(f'data/{csv_file}')
        
        all_targets = ordinal_targets + continuous_targets
        for target in all_targets:
            train_final_model(df, target, region)